In [2]:
# 🚀 Traffic ML App: Least Congested Route Finder (Uses Speed Dataset)

# 📂 Import libraries
import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import folium
from shapely.geometry import LineString, MultiLineString, Polygon, MultiPolygon

# 📂 Paths to datasets
road_geojson_path = r"C:\Users\sagni\Downloads\Road Traffic\archive (1)\new_delhi_traffic_dataset\geojson\new_delhi.json"
speed_csv_path = r"C:\Users\sagni\Downloads\Road Traffic\archive (1)\new_delhi_traffic_dataset\weekday_stats\2024_week_day_speed_city.csv"

# 📥 Load road network (GeoJSON)
roads_gdf = gpd.read_file(road_geojson_path)
print("✅ Road network loaded. Segments:", len(roads_gdf))

# 📥 Load speed dataset
speed_df = pd.read_csv(speed_csv_path)
print("✅ Speed dataset loaded. Shape:", speed_df.shape)

# 🧹 Clean speed dataset: strip " km/h" and convert to float
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in days:
    speed_df[day] = speed_df[day].str.replace(' km/h', '', regex=False).astype(float)

# 📅 User selects day and hour
day_of_week = 'Monday'   # Change as needed (Monday to Sunday)
hour_of_day = 9          # Change as needed (0-23)

# 🧮 Get average city speed for fallback
mean_city_speed = speed_df[day_of_week].mean()
print(f"📊 Average speed for {day_of_week}: {mean_city_speed:.2f} km/h")

# 🛠️ Build graph from GeoJSON (convert polygons to lines if needed)
G = nx.DiGraph()
converted = 0
for idx, row in roads_gdf.iterrows():
    geom = row['geometry']
    lines = []
    if isinstance(geom, LineString):
        lines = [geom]
    elif isinstance(geom, MultiLineString):
        lines = list(geom.geoms)
    elif isinstance(geom, Polygon):
        lines = [LineString(geom.exterior.coords)]
        converted += 1
    elif isinstance(geom, MultiPolygon):
        for poly in geom.geoms:
            lines.append(LineString(poly.exterior.coords))
        converted += 1
    else:
        continue  # skip unsupported geometries

    for line in lines:
        coords = list(line.coords)
        for i in range(len(coords) - 1):
            u = coords[i]
            v = coords[i + 1]
            length_m = line.length
            # Get speed for segment if available, else fallback
            speed_kmh = mean_city_speed  # Fallback for now
            travel_time_sec = (length_m / 1000) / speed_kmh * 3600
            G.add_edge(u, v, weight=travel_time_sec, length=length_m, speed_kmh=speed_kmh)
print(f"✅ Graph built. Nodes: {len(G.nodes)}, Edges: {len(G.edges)}, Polygons converted: {converted}")

# 📍 Define start and end points (lat, lon)
origin = (28.6139, 77.2090)      # Connaught Place
destination = (28.4595, 77.0266) # Gurgaon

# 🔎 Find nearest nodes in graph
def get_nearest_node(G, point):
    return min(G.nodes, key=lambda n: (n[0] - point[1])**2 + (n[1] - point[0])**2)

source_node = get_nearest_node(G, origin)
target_node = get_nearest_node(G, destination)

# 🛣️ Find least congested path
route = nx.shortest_path(G, source_node, target_node, weight='weight')
print("✅ Route found. Number of steps:", len(route))

# 🌍 Visualize route with Folium
m = folium.Map(location=origin, zoom_start=12)
route_coords = [(lat, lon) for lon, lat in route]
folium.PolyLine(route_coords, color="blue", weight=5, tooltip="Fastest Route").add_to(m)
folium.Marker(origin, popup="Start: Connaught Place", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination, popup="End: Gurgaon", icon=folium.Icon(color="red")).add_to(m)

# 💾 Save and display map
output_map = "new_delhi_fastest_route.html"
m.save(output_map)
print(f"✅ Map saved as {output_map}. Open it in your browser to view the route.")


✅ Road network loaded. Segments: 1
✅ Speed dataset loaded. Shape: (12, 8)
📊 Average speed for Monday: 30.00 km/h
✅ Graph built. Nodes: 11, Edges: 11, Polygons converted: 1
✅ Route found. Number of steps: 11
✅ Map saved as new_delhi_fastest_route.html. Open it in your browser to view the route.


In [3]:
# 🚀 Delhi Traffic ML App: Least Congested Route Finder (City-Wide)

# 📦 Install osmnx if not already installed
try:
    import osmnx as ox
except ImportError:
    !pip install osmnx
    import osmnx as ox

import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
import folium

# 📂 Path to your speed dataset
speed_csv_path = r"C:\Users\sagni\Downloads\Road Traffic\archive (1)\new_delhi_traffic_dataset\weekday_stats\2024_week_day_speed_city.csv"

# 📥 Load speed dataset
speed_df = pd.read_csv(speed_csv_path)
print("✅ Speed dataset loaded. Shape:", speed_df.shape)

# 🧹 Clean speed dataset: strip " km/h" and convert to float
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
for day in days:
    speed_df[day] = speed_df[day].str.replace(' km/h', '', regex=False).astype(float)

# 📅 User selects day and hour
day_of_week = 'Monday'   # Change as needed
hour_of_day = 9          # Change as needed (0-23)

# 🧮 Get average city speed for fallback
mean_city_speed = speed_df[day_of_week].mean()
print(f"📊 Average speed for {day_of_week}: {mean_city_speed:.2f} km/h")

# 🌐 Download full road network of Delhi from OpenStreetMap
print("🌐 Downloading Delhi road network from OSM...")
G = ox.graph_from_place("Delhi, India", network_type='drive')
print("✅ Delhi road network loaded. Nodes:", len(G.nodes), "Edges:", len(G.edges))

# 🏎️ Add speed limits to edges from speed_df (simulate mapping)
# For simplicity, assign mean city speed to all edges (OSM data may lack speed info)
for u, v, k, data in G.edges(keys=True, data=True):
    data['speed_kmh'] = mean_city_speed
    # Estimate travel time (seconds) = length (m) / speed (m/s)
    length_m = data.get('length', 100)  # fallback 100m if no length
    speed_ms = data['speed_kmh'] * 1000 / 3600
    data['travel_time_sec'] = length_m / speed_ms if speed_ms > 0 else 9999

print("✅ Speed data applied to OSM network edges.")

# 📍 Define origin and destination (lat, lon)
origin = (28.6139, 77.2090)      # Connaught Place
destination = (28.4595, 77.0266) # Gurgaon

# 🔎 Find nearest nodes in graph
orig_node = ox.distance.nearest_nodes(G, origin[1], origin[0])
dest_node = ox.distance.nearest_nodes(G, destination[1], destination[0])

# 🛣️ Compute least congested path (Dijkstra using travel_time_sec)
route = nx.shortest_path(G, orig_node, dest_node, weight='travel_time_sec')
print("✅ Route found. Number of steps:", len(route))

# 🌍 Visualize route with Folium
m = folium.Map(location=origin, zoom_start=11)
route_coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route]
folium.PolyLine(route_coords, color="blue", weight=5, tooltip="Fastest Route").add_to(m)
folium.Marker(origin, popup="Start: Connaught Place", icon=folium.Icon(color="green")).add_to(m)
folium.Marker(destination, popup="End: Gurgaon", icon=folium.Icon(color="red")).add_to(m)

# 💾 Save and display map
output_map = "delhi_least_congested_route.html"
m.save(output_map)
print(f"✅ Map saved as {output_map}. Open it in your browser to view the route.")


✅ Speed dataset loaded. Shape: (12, 8)
📊 Average speed for Monday: 30.00 km/h
🌐 Downloading Delhi road network from OSM...
✅ Delhi road network loaded. Nodes: 183023 Edges: 498303
✅ Speed data applied to OSM network edges.
✅ Route found. Number of steps: 182
✅ Map saved as delhi_least_congested_route.html. Open it in your browser to view the route.
